# Build a ChatBot 

We have built a simple question-answering pipeline, where we can input a question and get an answer. In addition to single round of question-answering, we can also have a conversation with an LLM by building a chatbot. The chatbot can remember the history of the conversation and respond based on the history. The key to achieve this is to leverage the promt args `chat_history_str` and the data structure `Memory` to manage the conversation history.

In [ ]:
from IPython.display import clear_output
!pip install -U adalflow[openai,groq,faiss-cpu]
clear_output()

In [ ]:
# Import needed modules from Adalflow
import os
from getpass import getpass
from adalflow.core.component import Component
from adalflow.core.generator import Generator
from adalflow.components.memory.memory import Memory
from adalflow.components.model_client import OpenAIClient  # Here, we use the OpenAIClient as an example, but you can use any other clients (with the corresponding API Key as needed), such as AnthropicAPIClient

In [ ]:
# Prompt user to enter their API keys securely
openai_api_key = getpass("Please enter your OpenAI API key: ")
# Set environment variables
os.environ['OPENAI_API_KEY'] = openai_api_key
# Replace with your OpenAI API Key, or you can put it in a .env file

In [ ]:
# Build the ChatBot pipeline
class ChatBot(Component):
    def __init__(self):
        super().__init__()
        self.generator = Generator(
            model_client=OpenAIClient(),
            model_kwargs={'model': 'gpt-4o-mini'}
        )
        self.chat_history = Memory() # Memory to store the chat history
    
    def call(self) -> str:
        print("Welcome to the ChatBot. Type anything to chat. Type 'exit' to end.")
        while True:
            user_input = input("You: ")
            if user_input.lower() == "exit":
                print("Goodbye!")
                break
            chat_history_str = self.chat_history()
            # Generate the response from the user input and chat history
            response = self.generator(
                prompt_kwargs={
                    "input_str": user_input,
                    "chat_history_str": chat_history_str,
                },
            )
            # Save the user input and response to the memory
            self.chat_history.add_dialog_turn(
                user_query=user_input, assistant_response=response
            )
            print(f"ChatBot: {response}")

chatbot = ChatBot()
print(chatbot)

In [ ]:
chatbot.call()